In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd

In [2]:
#routard
def scrap_routard(continent, country, city):
    """continent, country, city in FRENCH"""
    response = requests.get(f'https://www.routard.com/fr/guide/{continent}/{country}/{city}')
    routard_soup = BeautifulSoup(response.text, 'html.parser')
    links = routard_soup.find_all('a', attrs={'href': re.compile(f'guide/{continent}/{country}/{city}')})
    for link in links:
        print(link.get('href').split("/")[-1])
    # scraper les liens de chaque truc pour récup la description rapide ?

scrap_routard('europe', 'grece', 'athenes')

acropole
agora
erechtheion
jardin-national
mont-lycabette
musee-archeologique-national
musee-de-l-acropole
parlement-place-syndagma
parthenon
quartier-de-plaka


In [3]:
#guide michelin
region = 'attica'
city = 'athens'

In [6]:
COSTS = ['Affordable', 'Mid-Range', 'Premium', 'Luxury']

def scrap_michelin(city, region=None):
    data = []
    if region is None:
        url = f"https://guide.michelin.com/fr/fr/{city}-region/restaurants?sort=distance"
    else:
        url = f"https://guide.michelin.com/fr/fr/{region}/{city}/restaurants?sort=distance"
    response = requests.get(url)
    michelin_soup = BeautifulSoup(response.text, 'html.parser')
    restaurants = michelin_soup.find_all('div', {'class': "card__menu-content"})
    if len(restaurants)==0:
        return None
    for resto in restaurants:
        resto_a = resto.find('a')
        name = resto_a.text.strip()
        link = "https://guide.michelin.com" + resto_a.get('href')
        nb_stars = 0
        bib_gourmand = False #bib-gourmand: nos meilleurs rapports qualité-prix
        gastronomie_durable = False
        gastronomie_durable = False
        for span in resto.find_all('span', {'class':'distinction-icon'}):
            for distinction in span.find_all('img'):
                src = distinction.get('src')
                if "star" in src:
                    nb_stars += 1
                elif "bib-gourmand" in src:
                    bib_gourmand = True
                elif "gastronomie-durable" in src:
                    gastronomie_durable = True
    
        resto_div = resto.find_all('div', {'class': "card__menu-footer--score"})
        tmp = re.sub(r'\s+', ' ', resto_div[1].text).strip() # to get "<$/€> · <cuisine>"
        price, cuisine = tmp.split(" · ")
        cost = COSTS[len(price)-1]
        address, latitude, longitude = scrap_resto_link(link)
        data.append({
            "name": name,
            "link": link,
            "stars": nb_stars,
            "bib-gourmand": bib_gourmand,
            "cuisine": cuisine,
            "cost": cost,
            "address": address,
            "latitude":latitude,
            "longitude": longitude,
            "gastonomie_durable": gastronomie_durable
        })
    return pd.DataFrame(data)

def scrap_resto_link(url):
    resto_req = requests.get(url)
    resto_soup = BeautifulSoup(resto_req.text, 'html.parser')
    address = ""
    for div in resto_soup.find_all('div', {'class':'data-sheet__block--text'}):
        text = div.text.strip()
        if not '\n' in text:
            address = text
            break
    latitude, longitude = None, None
    for iframe in resto_soup.find_all('iframe'):
        i_src = iframe.get('src')
        if i_src is None:
            continue
        if "maps" in i_src:
            search = re.search(r'q=([-+]?[0-9]*\.?[0-9]+),([-+]?[0-9]*\.?[0-9]+)', i_src)
            if search:
                latitude = float(search.group(1))
                longitude = float(search.group(2))
    return address, latitude, longitude

display(scrap_michelin('athens', 'attica'))

,name,link,stars,bib-gourmand,cuisine,cost,price,address,latitude,longitude,gastonomie_durable
0,GB Roof Garden,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine méditerranéenne,Premium,€€€,"1A Vasileos Georgiou A street, Athènes, 105 64...",37.976192,23.735390,False
1,Tudor Hall,https://guide.michelin.com/fr/fr/attica/athens...,1,False,Cuisine actuelle,Luxury,€€€€,"3A Vasileos Georgiou A, Athènes, 10564, Grèce",37.976277,23.734933,False
2,Ovio,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine italienne contemporaine,Mid-Range,€€,"4 Apollonos, Athènes, 105557, Grèce",37.974820,23.732740,False
3,Nolan,https://guide.michelin.com/fr/fr/attica/athens...,0,True,Cuisine Fusion,Affordable,€,"31-33 Voulis Street, Athènes, 10557, Grèce",37.974844,23.732328,False
4,Aneton,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine méditerranéenne,Mid-Range,€€,"3 Navarchou Nikodimou Street, Athènes, 10 557,...",37.973477,23.733089,False
5,Okio,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine méditerranéenne,Mid-Range,€€,"3 Navarhou Nikodimou, Athènes, 10557, Grèce",37.973461,23.732977,False
6,The Zillers,https://guide.michelin.com/fr/fr/attica/athens...,1,False,Cuisine actuelle,Premium,€€€,"54 Mitropoleos, Athènes, 10563, Grèce",37.975705,23.729491,False
7,Dolli's,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine méditerranéenne,Premium,€€€,"49 Mitropoleos, Athènes, 10556, Grèce",37.975843,23.728085,False
8,Fine Mess Smokehouse,https://guide.michelin.com/fr/fr/attica/athens...,0,True,Barbecue,Mid-Range,€€,"5 Delfon Street, Athènes, 10680, Grèce",37.982257,23.736919,False
9,SENSE,https://guide.michelin.com/fr/fr/attica/athens...,0,False,Cuisine créative,Premium,€€€,"5 Dionysiou Areopagitou Street, Athènes, 117 4...",37.969501,23.730835,False


In [6]:
# https://guide.michelin.com/fr/fr/attica/athens/restaurants/page/2?sort=distance
# faire les pages suivantes ?

In [8]:
# https://guide.michelin.com/fr/fr/osaka-region/restaurants
region = "osaka"
city = "osaka"
data = scrap_michelin(city, region)
if data is None:
    data = scrap_michelin(region)


,name,link,stars,bib-gourmand,cuisine,cost,price,address,latitude,longitude,gastonomie_durable
0,YUNiCO,https://guide.michelin.com/fr/fr/osaka-region/...,1,False,Cuisine italienne,Premium,¥¥¥,"4F, 1-5-17 Sonezakishinchi, Kita-ku, Osaka, 53...",34.698070,135.496874,False
1,TAVERNETTA da KITAYAMA,https://guide.michelin.com/fr/fr/osaka-region/...,0,False,Cuisine italienne,Mid-Range,¥¥,"1-6 Ishigatsujicho, Tennoji-ku, Osaka, 543-003...",34.664805,135.523061,False
2,Chukasoba Mugen,https://guide.michelin.com/fr/fr/osaka-region/...,0,True,Spécialité Ramen,Affordable,¥,"5-5-10 Ebie, Fukushima-ku, Osaka, 553-0001, Japon",34.696615,135.472155,False
3,Sobakiri Arabompu,https://guide.michelin.com/fr/fr/osaka-region/...,0,True,Spécialités Soba,Affordable,¥,"4-1-11 Nishitemma, Kita-ku, Osaka, 530-0047, J...",34.696363,135.505318,False
4,Sobakiri Imose,https://guide.michelin.com/fr/fr/osaka-region/...,0,True,Spécialités Soba,Affordable,¥,"2-9-2 Tamatsukuri, Chuo-ku, Osaka, 540-0004, J...",34.676700,135.529687,False
5,Naniwa Okina,https://guide.michelin.com/fr/fr/osaka-region/...,0,True,Spécialités Soba,Affordable,¥,"4-1-18 Nishitemma, Kita-ku, Osaka, 530-0047, J...",34.696220,135.505954,False
6,Iwaki,https://guide.michelin.com/fr/fr/osaka-region/...,1,False,Cuisine japonaise,Premium,¥¥¥,"2F, 1-6-29 Sonezakishinchi, Kita-ku, Osaka, 53...",34.697482,135.499446,False
7,Sushi Harasho,https://guide.michelin.com/fr/fr/osaka-region/...,2,False,Sushi,Premium,¥¥¥,"3-30 Uenomiyacho, Tennoji-ku, Osaka, 543-0037,...",34.662195,135.519860,False
8,Kosai Fukumimi,https://guide.michelin.com/fr/fr/osaka-region/...,0,False,Cuisine chinoise,Mid-Range,¥¥,"2F, 1-1-8 Sonezakishinchi, Kita-ku, Osaka, 530...",34.696528,135.500302,False
9,La bonne tâche,https://guide.michelin.com/fr/fr/osaka-region/...,0,True,Cuisine française,Mid-Range,¥¥,"4-1-8 Nishitemma, Kita-ku, Osaka, 530-0047, Japon",34.696181,135.505178,False


In [9]:
df_michelin

## Suppr stop words 13 langues

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
from nltk.corpus import stopwords
nltk.download('stopwords')

language_dict = {
  'cs': 'czech',
  'da': 'danish',
  'nl': 'dutch',
  'en':'english',
  'et': 'estonian',
  'fi': 'finnish',
  'fr': 'french',
  'de': 'german',
  'el': 'greek',
  'it': 'italian',
  'no': 'norwegian',
  'pl': 'polish',
  'pt': 'portuguese',
  'ru': 'russian',
  'sl': 'slovene',
  'es': 'spanish',
  'sv': 'swedish',
  'tr': 'turkish'
}


def tokenize_without_stopwords(text):
  try:
    language = language_dict[detect(text)]
    stop_words = set(stopwords.words(language))
    filtered_text = " ".join([word for word in text.split() if word not in stop_words])
    return word_tokenize(filtered_text, language=language)

  except Exception as e:
    if len(str(e)) == 4:
      print("[WARNING] language not supported:", str(e))
    else:
      print("error", e)
    return word_tokenize(text)